# Data z tramvají DPO
### Zkoumání pro korelaci s realitou

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [32]:
def remove_leader(path: str):
    return path.split(".")[-1]

def convert_sig_unsig(number):
    if number < 32767:
        return number
    else:
        return number-65535

def tram_csv_parser(csv_path):
    # Load from csv
    df = pd.read_csv(csv_path, delimiter=";", skiprows=73, decimal=",")
    default_cols = list(df.columns)

    # Rename cols
    rename_dict = {}
    for col in df.columns:
        new_name = remove_leader(col)
        rename_dict[col] = new_name
    df.rename(columns=rename_dict, inplace=True)

    # New cols (calculated)
    # Time
    df.insert(2, "DateTime", pd.to_datetime(df["Date"] + ' ' + df["Time"], format="%d.%m.%Y %H:%M:%S:%f"))
    del df["Date"]
    del df["Time"]
    df["delta_T"] = df["DateTime"].diff(periods=-1).abs()

    # Trakcni menic 1
    df["tm_stejnoproud_trakce"] = df["AM_A_C1_14_IwTCU_ILF1"].apply(convert_sig_unsig)/10
    df["tm_stejnoproud_spotrebice"] = df["AM_A_C1_14_IwAUX_I_DClink"]*0.1
    df["tm_stridaproud_vystupu"] = df["AM_A_C1_14_IwAUX_I_rms"]*0.01
    df["tm_vstupni_napeti"] = df["AM_A_C1_14_IwTCU_UD1Voltage"]/10
    df["tm_tah_vyvijeny_motory"] = df["AM_B_C1_15_IwTCU_E_MC1"].apply(convert_sig_unsig)/100
    df["tm_rychlost_3_napravy"] = df["VrTCU_A_A1_Velocity"]/3.6
    df["tm_prikon_trakce"] = df["tm_stejnoproud_trakce"]*df["tm_vstupni_napeti"]/1000
    df["tm_vykon_trakce"] = df["tm_tah_vyvijeny_motory"]*df["tm_rychlost_3_napravy"]
    df["tm_prikon_spotrebice"] = df["AM_A_C1_14_IwTCU_P_AUX"]/100
    df["tm_prikon_topeni_klima"] = 1.73*400*df["tm_stridaproud_vystupu"]/1000
    df["tm_vykon_zmareny"] = np.where((df["tm_vykon_trakce"] > 0) & (df["tm_vykon_trakce"] < df["tm_prikon_trakce"]), df["tm_prikon_trakce"]-df["tm_vykon_trakce"], 0)

    # Trakcni menic 2
    df["tm2_stejnoproud_trakce"] = df["AM_B_C1_15_IwTCU_ILF1"].apply(convert_sig_unsig)/10
    df["tm2_vstupni_napeti"] = df["AM_B_C1_15_IwTCU_UD1Voltage"]/10
    df["tm2_tah_vyvijeny_motory"] = df["AM_B_C1_15_IwTCU_E_MC1"].apply(convert_sig_unsig)/100
    df["tm2_rychlost_5_napravy"] = df["VrTCU_B_A1_Velocity"]/3.6
    df["tm2_prikon_trakce"] = df["tm2_stejnoproud_trakce"]*df["tm2_vstupni_napeti"]/1000
    df["tm2_vykon_trakce"] = df["tm2_tah_vyvijeny_motory"]*df["tm2_rychlost_5_napravy"]
    df["tm2_vykon_zmareny"] = np.where((df["tm2_vykon_trakce"] > 0) & (df["tm2_vykon_trakce"] < df["tm2_prikon_trakce"]), df["tm2_prikon_trakce"]-df["tm2_vykon_trakce"], 0)

    # Cela tramvaj
    df["cela_vykon_spotrebovany"] = np.where(df["tm_prikon_trakce"]+df["tm_prikon_spotrebice"]+df["tm2_prikon_trakce"] > 0, df["tm_prikon_trakce"]+df["tm_prikon_spotrebice"]+df["tm2_prikon_trakce"], 0)
    df["cela_vykon_rekuperovany"] = np.where(df["tm_prikon_trakce"]+df["tm_prikon_spotrebice"]+df["tm2_prikon_trakce"] < 0, -(df["tm_prikon_trakce"]+df["tm_prikon_spotrebice"]+df["tm2_prikon_trakce"]), 0)
    df["cela_vykon_zmareny"] = df["tm_vykon_zmareny"]+df["tm2_vykon_zmareny"]
    df["cela_vykon_trakce"] = df["tm_vykon_trakce"]+df["tm2_vykon_trakce"]
    df["cela_prikon_topeni_klima"] = df["tm_prikon_topeni_klima"]

    df["cela_energie_spotrebovana"] = (df["cela_vykon_spotrebovany"]*df["delta_T"].dt.total_seconds()/3600).cumsum().shift(1)
    df.loc[0, "cela_energie_spotrebovana"] = 0.0

    df["cela_energie_rekuperovana"] = (df["cela_vykon_rekuperovany"]*df["delta_T"].dt.total_seconds()/3600).cumsum().shift(1)
    df.loc[0, "cela_energie_rekuperovana"] = 0.0

    df["cela_energie_zmarena"] = (df["cela_vykon_zmareny"]*df["delta_T"].dt.total_seconds()/3600).cumsum().shift(1)
    df.loc[0, "cela_energie_zmarena"] = 0.0

    df["cela_energie_topeni_klima"] = (df["cela_prikon_topeni_klima"]*df["delta_T"].dt.total_seconds()/3600).cumsum().shift(1)
    df.loc[0, "cela_energie_topeni_klima"] = 0.0

    df["km_total"] = df["QdwMesitTotalDistance"]*0.1

    return df
    return default_cols

tram_csv_parser("../testing-data/DPO/Jízda_Poruba_Zátiší_20.04.2022/02-Vresinska-Zatisi_2022-04-20.csv")

,Nb,Type,DateTime,AM_A_C1_14_IwAUX_I_DClink,AM_A_C1_14_IwAUX_I_rms,AM_A_C1_14_IwTCU_E_MC1,AM_A_C1_14_IwTCU_ILF1,AM_A_C1_14_IwTCU_P_AUX,AM_A_C1_14_IwTCU_UD1Voltage,AM_B_C1_15_IwTCU_E_MC1,...,cela_vykon_spotrebovany,cela_vykon_rekuperovany,cela_vykon_zmareny,cela_vykon_trakce,cela_prikon_topeni_klima,cela_energie_spotrebovana,cela_energie_rekuperovana,cela_energie_zmarena,cela_energie_topeni_klima,km_total
0,0,B+,2022-04-20 09:07:04.479,387,3765,697,466,2661,6218,693,...,83.68672,0.0,6.628284,50.448436,26.05380,0.000000,0.000000,0.000000,0.000000,239127.1
1,1,+,2022-04-20 09:07:04.577,389,3759,588,387,2647,6226,586,...,74.90046,0.0,5.804967,42.625494,26.01228,0.002278,0.000000,0.000180,0.000709,239127.1
2,2,+,2022-04-20 09:07:04.678,388,3761,434,250,2644,6258,440,...,57.79760,0.0,0.000000,32.029604,26.02612,0.004380,0.000000,0.000343,0.001439,239127.1
3,3,+,2022-04-20 09:07:04.778,382,3763,192,65,2644,6306,191,...,34.44986,0.0,0.000000,13.746657,26.03996,0.005985,0.000000,0.000343,0.002162,239127.1
4,4,+,2022-04-20 09:07:04.878,382,3762,0,36,2643,6315,0,...,31.10272,0.0,0.000000,0.000000,26.03304,0.006942,0.000000,0.000343,0.002885,239127.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12462,12462,+,2022-04-20 09:27:50.678,300,3208,56,59,2253,6787,60,...,30.33449,0.0,3.540188,4.264302,22.19936,33.521805,2.771225,3.509791,10.731072,239136.1
12463,12463,+,2022-04-20 09:27:50.778,301,3208,59,61,2253,6784,60,...,30.60238,0.0,3.801298,4.271082,22.19936,33.522647,2.771225,3.509890,10.731689,239136.1
12464,12464,+,2022-04-20 09:27:50.878,300,3207,62,58,2253,6779,62,...,30.25862,0.0,3.316917,4.411703,22.19244,33.523497,2.771225,3.509995,10.732305,239136.1
12465,12465,+,2022-04-20 09:27:50.978,302,3200,62,55,2249,6735,59,...,29.76751,0.0,3.077600,4.199910,22.14400,33.524338,2.771225,3.510087,10.732922,239136.1
